In [2]:
import pandas as pd
import sqlite3

## create a connection to the database 

In [3]:
conn= sqlite3.connect('../data/checking-logs.sqlite')

## using only one query for each of the groups, create two dataframes: test_results and control_results with the columns time and avg_diff and only two rows
time should have the values: after and before;

avg_diff contains the average delta among all the users for the time period before each of them made their first visit to the page and afterward;

only take into account the users that have observations before and after;

In [7]:
query = """
WITH user_diffs AS (
    SELECT 
        uid,
        CASE 
            WHEN first_commit_ts < first_view_ts THEN 'before'
            ELSE 'after' 
        END AS time,
        CAST((julianday(first_commit_ts) - julianday(deadlines.deadlines, 'unixepoch')) * 24 AS INTEGER) AS diff
    FROM 
        test
    LEFT JOIN deadlines ON test.labname = deadlines.labs
    WHERE 
        test.labname != 'project1'
)
SELECT 
    time,
    AVG(diff) AS avg_diff
FROM user_diffs
WHERE uid IN (
    SELECT uid
    FROM user_diffs
    GROUP BY uid
    HAVING COUNT(DISTINCT time) = 2
)
GROUP BY time
"""
test_results = pd.read_sql(query, conn)
test_results

,time,avg_diff
0,after,-104.6000
1,before,-60.5625


In [8]:
query =  """
WITH user_diffs AS (
    SELECT 
        uid,
        CASE 
            WHEN control.first_commit_ts < control.first_view_ts THEN 'before'
            ELSE 'after' 
        END AS time,
        CAST((julianday(control.first_commit_ts) - julianday(deadlines.deadlines, 'unixepoch')) * 24 AS INTEGER) AS diff
    FROM 
        control
    LEFT JOIN deadlines ON control.labname = deadlines.labs
    WHERE 
        control.labname != 'project1'
)
SELECT 
    time,
    AVG(diff) AS avg_diff
FROM user_diffs
WHERE uid IN (
    SELECT uid
    FROM user_diffs
    GROUP BY uid
    HAVING COUNT(DISTINCT time) = 2
)
GROUP BY time
"""
control_results = pd.read_sql(query, conn)
control_results

,time,avg_diff
0,after,-117.636364
1,before,-99.464286


In [9]:
conn.close()